In [3]:
# process Call Rpt bulk data files  (major fixes needed to RCR schedule 20160507)
# WARNING: Requires that the Call Report bulk data files have been downloaded and unzipped
#          and placed in directories named C:/BankRisk/Bulk200103 (for March 2001 files as an example)
#          If there are any missing files, there will be error messages and the code will stop running

import numpy as np
import pandas as pd
from datetime import datetime
from time import strftime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')
import os

os.chdir('C:/BankRisk')  # set directory for working data files here 

In [4]:
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2016-05-07 07:57:41


In [71]:
# requires that all 4 quarters of any year be available:
yst = 2010  # start year of bulk data to process
yen = 2016  # end year to process (Note: python stops at 1 less than requested value!! if yen = 2010, will stop at 2009!!)

yrs = np.arange(yst,yen)
mns = ['03', '06', '09', '12']
mds = ['0331', '0630', '0930','1231']
dmd = dict(zip(mns, mds))
dys = [31,30,30,31]
ddd = dict(zip(mns,dys))

yrmn_beg = str(min(yrs)) + '03'

for y in yrs:
  for m in mns:
    asof = dmd[m] + str(y)
    yrmn = str(y) + m

    n = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Bulk POR "+asof+".txt", sep='\t', header=0, index_col=0)
    n.replace(to_replace='NATIONAL ASSOCIATION', value='NA', inplace=True, regex=True)

    nc = ['Cert','OCCchtr','OTSdckt','ABArt','Name','Address','City','State','Zip','FilTyp','SubmtDt']
    n.columns = nc

    # de-dup file:
    n = n.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    df = n[['Name','State','Cert']]

    ##########################################
    # process schedule RC:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RC "+asof+".txt", sep='\t', header=0, index_col=0)
    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    #x.drop('RCON8678', axis=1, inplace=True)  # remove fiscal year-end date ("12-31")
    x = x.convert_objects(convert_numeric=True)
    x = x.fillna(0)

    # process raw data from schedule:
    df['eTotAst'] = x.apply(lambda x: max(x['RCON2170'],x['RCFD2170']),axis=1)
    df['eTotLn'] = x.apply(lambda x: max(x['RCON5369'],x['RCFD5369'])+max(x['RCONB528'],x['RCFDB528']),axis=1)
    df['eALLL'] = x.apply(lambda x: max(x['RCON3123'],x['RCFD3123']), axis=1)
    df['eOREO'] = x.apply(lambda x: max(x['RCON2150'],x['RCFD2150']), axis=1)
    df['eGW'] = x.apply(lambda x: max(x['RCON3163'],x['RCFD3163']), axis=1)
    df['eOthInt'] = x.apply(lambda x: max(x['RCON0426'],x['RCFD0426']), axis=1)

    try:
      df['eTotEq'] = x.apply(lambda x: max(x['RCONG105'],x['RCFDG105']), axis=1)
    except:
      df['eTotEq'] = x.apply(lambda x: max(x['RCON3210'],x['RCFD3210']), axis=1)

    df['eTangEq'] = df['eTotEq'] - df['eGW'] - df['eOthInt']
    df['eDDep'] = x['RCON2200']
    df['eDDepNIB'] = x['RCON6631']
    df['eDDepIB'] = x['RCON6636']
    df['eFDep'] = x['RCFN2200']
    df['eFDepNIB'] = x['RCFN6631']
    df['eFDepIB'] = x['RCFN6636']
    df['eDDA'] = df['eDDepNIB']+df['eFDepNIB']

    ################################################
    # process schedule RCK: qtrly average balances:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCK "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    x = x.convert_objects(convert_numeric=True)
    x=x.fillna(0)

    df['aTotAst'] = x.apply(lambda x: max(x['RCON3368'],x['RCFD3368']),axis=1)
    df['aDLn'] = x['RCON3360']

    try:
      df['aSFR'] = x['RCON3465']
      df['aRE'] = x['RCON3466']
      df['aTotRE'] = x['RCON3465'] + x['RCON3466']
    except:
      df['aSFR'] = 0
      df['aRE'] = 0
      df['aTotRE'] = x['RCON3385']

    df['aAg'] = x['RCON3386']
    df['aCI'] = x['RCON3387']
    df['aCard'] = x['RCONB561']
    df['aOthCsmr'] = x['RCONB562']
    df['aFLn'] = x['RCFN3360']
    df['aLeas'] = x.apply(lambda x: max(x['RCON3484'],x['RCFD3484']), axis=1)
    df['aTrad'] = x.apply(lambda x: max(x['RCON3401'],x['RCFD3401']), axis=1)

    df['aDFIB'] = x.apply(lambda x: max(x['RCON3381'],x['RCFD3381']), axis=1)
    df['aTreas'] = x.apply(lambda x: max(x['RCONB558'],x['RCFDB558']), axis=1)
    df['aMBS'] = x.apply(lambda x: max(x['RCONB559'],x['RCFDB559']), axis=1)
    df['aOSec'] = x.apply(lambda x: max(x['RCONB560'],x['RCFDB560']), axis=1)
    df['aFFS'] = x.apply(lambda x: max(x['RCON3365'],x['RCFD3365']), axis=1)
    
    df['aTotLn'] = x['RCON3360'] + x['RCFN3360']
    df['aTotLnLs'] = df['aTotLn'] + df['aLeas']  # 20160320
    df['aOthLnLs'] = df['aTotLnLs'] - df['aCI'] - df['aTotRE'] - df['aCard'] - df['aOthCsmr']  # 20160320
    
    df['aTotSec'] = df['aTreas'] + df['aMBS'] + df['aOSec']
    df['aOAstIB'] = df['aFFS'] + df['aDFIB'] +df['aTrad']
    df['aEarnAst'] = df['aTotLn']+df['aTrad']+df['aDFIB']+df['aTreas']+df['aMBS']+\
                     df['aOSec']+df['aFFS']+df['aLeas']

    df['aNOW'] = x['RCON3485']
    df['aSav'] = x['RCONB563']
    df['aCDun'] = x['RCONA529']
    df['aCDov'] = x['RCONA514']
    df['aDDepIB'] = df['aNOW'] + df['aSav'] + df['aCDun'] + df['aCDov']
    df['aFDepIB'] = x['RCFN3404']
    df['aFFP'] = x.apply(lambda x: max(x['RCON3353'],x['RCFD3353']), axis=1)
    df['aBorr'] = x.apply(lambda x: max(x['RCON3355'],x['RCFD3355']), axis=1)
    df['aDepIB'] = df['aDDepIB']+df['aFDepIB']
    df['aOLbIB'] = df['aFFP'] + df['aBorr']
    
    ########################################################
    # process schedule RCCI: ending loan balances:
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCCI "+asof+".txt", sep='\t', header=0, index_col=0)
    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

    x = x.convert_objects(convert_numeric=True)
    x = x.fillna(0)

    # process raw data from schedule RC-CI:
    df['eTotLn'] = x.apply(lambda x: max(x['RCON2122'],x['RCFD2122']),axis=1)
    df['eDLn'] = x['RCON2122']

    try:
      df['eCxSFR'] = x['RCONF158']
      df['eCxOth'] = x['RCONF159']
      df['eCx'] = x['RCONF158'] + x['RCONF159']
    except:
      df['eCxSFR'] = 0
      df['eCxOth'] = 0
      df['eCx'] = x['RCON1415']

    df['eFarm'] = x['RCON1420']
    df['eSfRev'] = x['RCON1797']
    df['eSfFst'] = x['RCON5367']
    df['eSfSec'] = x['RCON5368']
#    df['eSfTD'] = x['RCON1797'] + x['RCON5367']  # added 6/10/15. Needed for loss vol calcs
    df['eSfTD'] = x['RCON5367'] + x['RCON5368']  # corrected 8/18/15 from prior line
    df['eMult'] = x['RCON1460']

    try:
      df['eOwOc'] = x['RCONF160']
      df['eOthCRE'] = x['RCONF161']
      df['eCRE'] = x['RCONF160'] + x['RCONF161']
    except:
      df['eOwOc'] = 0
      df['eOthCRE'] = 0
      df['eCRE'] = x['RCON1480'] 

    df['eCREtot'] = df['eCx'] + df['eMult'] + df['eCRE']
    df['eFinInst'] = x.apply(lambda x: max(x['RCON1288'],(x['RCFDB532']+x['RCFDB533']+\
                            x['RCFDB534']+x['RCFDB536']+x['RCFDB537'])),axis=1)
    df['eAg'] = x.apply(lambda x: max(x['RCON1590'],x['RCFD1590']),axis=1)
    df['eCIdom'] = x.apply(lambda x: max(x['RCON1763'],x['RCFD1763']),axis=1)
    df['eCIfor'] = x.apply(lambda x: max(x['RCON1764'],x['RCFD1764']),axis=1)
    df['eCard'] = x.apply(lambda x: max(x['RCONB538'],x['RCFDB538']),axis=1)
    df['eOthRev'] = x.apply(lambda x: max(x['RCONB539'],x['RCFDB539']),axis=1)
    try:
      df['eAuto'] = x.apply(lambda x: max(x['RCONK137'],x['RCFDK137']),axis=1)
      df['eOthCsmr'] = x.apply(lambda x: max(x['RCONK207'],x['RCFDK207']),axis=1)
    except:
      df['eAuto'] = 0
      df['eOthCsmr'] = x.apply(lambda x: max(x['RCON2011'],x['RCFD2011']),axis=1)

    df['eForGovt'] = x.apply(lambda x: max(x['RCON2081'],x['RCFD2081']),axis=1)
    #df['eOthLn'] = x.apply(lambda x: max((x['RCONJ454']+x['RCONJ464']),(x['RCFD2107']+x['RCFD1563'])),axis=1)

    try:
      df['eLeas'] = x.apply(lambda x: max(x['RCON2165'],(x['RCFDF162']+x['RCFDF163'])),axis=1)
    except:
      df['eLeas'] = x.apply(lambda x: max(x['RCON2165'],(x['RCFD2182']+x['RCFD2183'])),axis=1)

    df['eUnearn'] = x.apply(lambda x: max(x['RCON2123'],x['RCFD2123']),axis=1)

    ###########################################################
    # process schedule RCN: non-accrual loans (need both parts)
    try:
      x1 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
      x1= x1.ix[x1.index[1:]]  # removes first data row containing col descriptions
      x1 = x1.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
  
      x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+"(2 of 2).txt", sep='\t', header=0, index_col=0)
      x2= x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
      x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')

      x = x1.merge(x2, how='inner', left_index=True, right_index=True)
    except:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCN "+asof+".txt", sep='\t', header=0, index_col=0)
      x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
      x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.convert_objects(convert_numeric=True)
    x = x.fillna(0)

    # process nonaccrual loans from schedule RC-N:

    try:
      df['nCxSFR'] = x['RCONF176']
      df['nCxOth'] = x['RCONF177']
      df['nCx'] = x['RCONF176'] + x['RCONF177']
    except:
      df['nCxSFR'] = 0
      df['nCxOth'] = 0
      df['nCx'] = x['RCON3492']

    df['nFarm'] = x['RCON3495']
    df['nSfRev'] = x['RCON5400']

    try:
      df['nSfFst'] = x['RCONC229']
      df['nSfSec'] = x['RCONC230']
      df['nSfTD'] = x['RCONC229'] + x['RCONC230']
    except:
      df['nSfFst'] = 0
      df['nSfSec'] = 0
      df['nSfTD'] = x['RCON5403']

    df['nMult'] = x['RCON3501']

    try:
      df['nOwOc'] = x['RCONF182']
      df['nOthCRE'] = x['RCONF183']
      df['nCRE'] = x['RCONF182'] + x['RCONF183']
    except:
      df['nOwOc'] = 0
      df['nOthCRE'] = 0
      df['nCRE'] = x['RCON3504']

    df['nCREtot'] = df['nCx'] + df['nMult'] + df['nCRE']
    df['nREfor'] = x['RCFNB574']
    df['nFinInst'] = x.apply(lambda x: max(x['RCONB836'],(x['RCFD5379']+x['RCFD5382'])),axis=1)
    df['nAg'] = x['RCFD1583']
    df['nCIdom'] = x.apply(lambda x: max(x['RCON1608'],x['RCFD1253']),axis=1)
    df['nCIfor'] = x['RCFD1256']
    df['nCard'] = x.apply(lambda x: max(x['RCONB577'],x['RCFDB577']),axis=1)
    try:
      df['nAuto'] = x.apply(lambda x: max(x['RCONK215'],x['RCFDK215']),axis=1)
      df['nOthCsmr'] = x.apply(lambda x: max(x['RCONK218'],x['RCFDK218']),axis=1)
    except:
      df['nAuto'] = 0
      df['nOthCsmr'] = x.apply(lambda x: max(x['RCONB580'],x['RCFDB580']),axis=1)

    df['nForGovt'] = x.apply(lambda x: max(x['RCON5391'],x['RCFD5391']),axis=1)
    df['nOthLn'] = x.apply(lambda x: max(x['RCON5461'],x['RCFD5461']),axis=1)

    try:
      df['nLeas'] = x.apply(lambda x: max(x['RCON1228'],(x['RCFDF168']+x['RCFDF171'])),axis=1)
    except:
      df['nLeas'] = x.apply(lambda x: max(x['RCON1228'],(x['RCFD1259']+x['RCFD1791'])),axis=1)

    df['nTotLn'] = df['nCx']+df['nFarm']+df['nSfRev']+df['nSfTD']+df['nMult']+\
                   df['nCRE']+df['nREfor']+df['nFinInst']+df['nAg']+df['nCIdom']+df['nCIfor']+\
                   df['nCard']+df['nAuto']+df['nOthCsmr']+df['nForGovt']+df['nOthLn']+df['nLeas']


    ###################################################################
    # process schedule RCE: (misc items)
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCE "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    
    x = x.convert_objects(convert_numeric=True)
    x=x.fillna(0)

    # process raw data from schedule:

    df['eBrok'] = x['RCON2365']
    df['eMMDA'] = x['RCON6810']
    df['eOthSav'] = x['RCON0352']
    try:
      df['eCDov250'] = x['RCONJ474']
    except:
      df['eCDov250'] = 0

    ###################################################################
    # process schedule RCO: (more misc items)
    try:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCO "+asof+".txt", sep='\t', header=0, index_col=0)
    except:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCO "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)  

    x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.convert_objects(convert_numeric=True)
    x = x.fillna(0)

    try:
      df['eDepov250'] = x['RCONF051']
      df['eIRAov250'] = x['RCONF047']
      df['eUninsDep'] = x['RCON5597']
    except:
      df['eDepov250'] = 0
      df['eIRAov250'] = 0
      df['eUninsDep'] = 0

    #####################################################################
    # process schedule RCR: regulatory capital

    try:
      x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCR "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
      x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
      x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
      x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCR "+asof+"(2 of 2).txt", sep='\t', header=0, index_col=0)
      x2 = x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
      x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
      x = x.merge(x2, how='inner', left_index=True, right_index=True)
    except:
      try:
        x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRIA "+asof+".txt", sep='\t', header=0, index_col=0)
        x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
        x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x2 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRIB "+asof+".txt", sep='\t', header=0, index_col=0)
        x2 = x2.ix[x2.index[1:]]  # removes first data row containing col descriptions
        x2 = x2.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x3 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRII "+asof+"(1 of 2).txt", sep='\t', header=0, index_col=0)
        x3 = x3.ix[x3.index[1:]]  # removes first data row containing col descriptions
        x3 = x3.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x = x.merge(x2, how='inner', left_index=True, right_index=True)
        x = x.merge(x3, how='inner', left_index=True, right_index=True)
      except:
        x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRI "+asof+".txt", sep='\t', header=0, index_col=0)
        x = x.ix[x.index[1:]]  # removes first data row containing col descriptions
        x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x3 = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RCRII "+asof+"(1 of 3).txt", sep='\t', header=0, index_col=0)
        x3 = x3.ix[x3.index[1:]]  # removes first data row containing col descriptions
        x3 = x3.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
        x = x.merge(x3, how='inner', left_index=True, right_index=True)

    x = x.apply(lambda z: pd.to_numeric(z, errors='coerce'))  ## 20160507
    #x = x.convert_objects(convert_numeric=True)  # deprecated with v0.18
    x = x.fillna(0)
    if 'RCON7206' not in x.columns:  ## 20160507: these cols were removed by FDIC in 2015
      x['RCON7206'] = 0
      x['RCON7275'] = 0
      x['RCON8274'] = 0
      x['RCON3792'] = 0
      x['RCONA223'] = 0
      x['RCONL138'] = 0

    try:
      df['eT1rbc'] = x.apply(lambda x: max(x['RCON7206'],x['RCFA7206'],x['RCFW7206']), axis=1)
      df['eTotrbc'] = x.apply(lambda x: max(x['RCON7275'],x['RCFA7205'],x['RCFW7205']), axis=1)
    except:
      df['eT1rbc'] = x.apply(lambda x: max(x['RCON7206'],x['RCFD7206'],x['RCFD7274']), axis=1)
      df['eTotrbc'] = x.apply(lambda x: max(x['RCON7275'],x['RCFD7205'],x['RCFD7275']), axis=1)

    try:
      df['eT1Cap'] = x.apply(lambda x: max(x['RCON8274'],x['RCFA8274']), axis=1)
      df['eTotCap'] = x.apply(lambda x: max(x['RCON3792'],x['RCFA3792']), axis=1)
      df['eTotRWA'] = x.apply(lambda x: max(x['RCONA223'],x['RCFAA223']), axis=1)
    except:
      df['eT1Cap'] = x.apply(lambda x: max(x['RCON8274'],x['RCFD8274']), axis=1)
      df['eTotCap'] = x.apply(lambda x: max(x['RCON3792'],x['RCFD3792']), axis=1)
      df['eTotRWA'] = x.apply(lambda x: max(x['RCONA223'],x['RCFDA223']), axis=1)

    try:
      df['eTotAstLev'] = x.apply(lambda x: max(x['RCONL138'],x['RCFAA224']), axis=1)
    except:
      try:
        df['eTotAstLev'] = x.apply(lambda x: max(x['RCONL138'],x['RCFDL138']), axis=1)
      except:
        df['eTotAstLev'] = x.apply(lambda x: max(x['RCONA224'],x['RCFDA224']), axis=1)


    #####################################################################
    # process schedule RI: income statement
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RI "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions

    #x.drop('RIADA530', axis=1, inplace=True)
    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    x = x.convert_objects(convert_numeric=True)
    x=x.fillna(0)

    try:
      df['iiSFR'] = x['RIAD4435']
      df['iiOthRE'] = x['RIAD4436']
      df['iiRE'] = x['RIAD4435'] + x['RIAD4436']
    except:
      df['iiSFR'] = 0
      df['iiOthRE'] = 0
      df['iiRE'] = x['RIAD4011']

    df['iiAg'] = x['RIAD4024']
    df['iiCI'] = x['RIAD4012']
    df['iiCard'] = x['RIADB485']
    df['iiOthCsmr'] = x['RIADB486']
    df['iiForGovt'] = x['RIAD4056']
    df['iiOthDomLn'] = x.apply(lambda x: max(x['RIAD4058'],x['RIADB487']), axis=1)
    df['iiFLn'] = x['RIAD4059']
    df['iiTotLn'] = x['RIAD4010']
    df['iiLeas'] = x['RIAD4065']
    df['iiDFIB'] = x['RIAD4115']
    df['iiTreas'] = x['RIADB488']
    df['iiMBS'] = x['RIADB489']
    df['iiOSec'] =x['RIAD4060']
    df['iiTrad'] =x['RIAD4069']
    df['iiFFS'] = x['RIAD4020']
    df['iiOthII'] = x['RIAD4518']
    df['iiTotLnLs'] = df['iiTotLn']+df['iiLeas']
    df['iiOthLnLs'] = df['iiTotLnLs'] - df['iiRE'] - df['iiCI'] - df['iiCard'] - df['iiOthCsmr'] # 20160320
    df['iiTotSec'] = df['iiTreas']+df['iiMBS']+df['iiOSec']
    df['iiOAstIB'] = df['iiDFIB']+df['iiTrad']+df['iiFFS']+df['iiOthII']
    df['iiTotII_pc'] = df['iiTotLnLs']+df['iiTotSec']+df['iiOAstIB']
    df['iiTotII'] = x['RIAD4107']

    df['ieNOW'] = x['RIAD4508']
    df['ieSav'] = x['RIAD0093']
    df['ieCDun'] = x['RIADA518']
    df['ieCDov'] = x['RIADA517']
    df['ieFDepIB'] = x['RIAD4172']
    df['ieFFP'] = x['RIAD4180']
    df['ieOthBorr'] = x['RIAD4185']
    df['ieNote'] = x['RIAD4200']
    df['ieDepIB'] = df['ieNOW'] + df['ieSav'] + df['ieCDun'] + df['ieCDov'] + df['ieFDepIB']
    df['ieOLbIB'] = df['ieFFP'] + df['ieOthBorr'] + df['ieNote']
    df['ieDDA'] = 0
    df['ieTotIE_pc'] = df['ieDepIB'] + df['ieOLbIB']
    df['ieTotIE'] = x['RIAD4073']
    df['iiNetII'] = x['RIAD4074']
    
    try:
      df['isIntangExp'] = x['RIADC216'] + x['RIADC232']
    except:
      df['isIntangExp'] = x['RIAD4531']

    df['isProv'] = x['RIAD4230']

    df['isTotNII'] = x['RIAD4079']
    df['isTotNIE'] = x['RIAD4093']
    df['isPreTax'] = x['RIAD4301']
    df['isNetInc'] = x['RIAD4340']

    ## added 10/31/15:  gain/loss items:
    df['isOREOgl'] = x['RIAD5415']
    df['isLnLsgl'] = x['RIAD5416']
    df['isOthAstgl'] = x['RIADB496']
    df['isHTMgl'] = x['RIAD3521']
    df['isAFSgl'] = x['RIAD3196']
    
    #########################################################################
    # process schedule RIBI: net charge-offs
    x = pd.read_csv("./Bulk"+yrmn+"/FFIEC CDR Call Schedule RIBI "+asof+".txt", sep='\t', header=0, index_col=0)
    x= x.ix[x.index[1:]]  # removes first data row containing col descriptions

    x = x.reset_index().drop_duplicates(['IDRSSD'],keep='last').set_index('IDRSSD')
    #x.drop('RIADA530', axis=1, inplace=True)
    x=x.astype(float)
    x=x.fillna(0)

    df['ncoTotLn'] = x['RIAD4635'] - x['RIAD4605']

    try:
      df['ncoCxSFR'] = x['RIADC891'] - x['RIADC892']
      df['ncoCxOth'] = x['RIADC893'] - x['RIADC894']
      df['ncoCx'] = x['RIADC891'] - x['RIADC892'] + x['RIADC893'] - x['RIADC894']
    except:
      df['ncoCxSFR'] = 0
      df['ncoCxOth'] = 0
      df['ncoCx'] = x['RIAD3582'] - x['RIAD3583']

    df['ncoFarm'] = x['RIAD3584'] - x['RIAD3585']
    df['ncoSfRev'] = x['RIAD5411'] - x['RIAD5412']

    try:
      df['ncoSfFst'] = x['RIADC234'] - x['RIADC217']
      df['ncoSfSec'] = x['RIADC235'] - x['RIADC218']
      df['ncoSfTD'] = x['RIADC234'] - x['RIADC217'] + x['RIADC235'] - x['RIADC218']
    except:
      df['ncoSfFst'] = 0
      df['ncoSfSec'] = 0
      df['ncoSfTD'] = x['RIAD5413'] - x['RIAD5414'] 

    df['ncoMult'] = x['RIAD3588'] - x['RIAD3589']

    try:
      df['ncoOwOc'] = x['RIADC895'] - x['RIADC896']
      df['ncoOthCRE'] = x['RIADC897'] - x['RIADC898']
      df['ncoCRE'] = x['RIADC895'] - x['RIADC896'] + x['RIADC897'] - x['RIADC898']
    except:
      df['ncoOwOc'] = 0
      df['ncoOthCRE'] = 0
      df['ncoCRE'] = x['RIAD3590'] - x['RIAD3591']

    df['ncoCREtot'] = df['ncoCx'] + df['ncoMult'] + df['ncoCRE']
    df['ncoREfor'] = x['RIADB512'] - x['RIADB513']
    df['ncoFIdom'] = x.apply(lambda x: max((x['RIAD4481']-x['RIAD4654']),x['RIAD4653']) -\
                             max((x['RIAD4482']-x['RIAD4664']),x['RIAD4663']),axis=1)
    df['ncoFIfor'] = x['RIAD4654'] - x['RIAD4664']
    df['ncoAg'] = x['RIAD4655'] - x['RIAD4665']
    df['ncoCIdom'] = x.apply(lambda x: max((x['RIAD4638']-x['RIAD4646']),x['RIAD4645']) -\
                             max((x['RIAD4608']-x['RIAD4618']),x['RIAD4617']),axis=1)
    df['ncoCIfor'] = x['RIAD4646'] - x['RIAD4618']
    df['ncoCard'] = x['RIADB514'] - x['RIADB515']
    try:
      df['ncoAuto'] = x['RIADK129'] - x['RIADK133']
      df['ncoOthCsmr'] = x['RIADK205'] - x['RIADK206']
    except:
      df['ncoAuto'] = 0
      df['ncoOthCsmr'] = x['RIADB516'] - x['RIADB517']

    df['ncoForGovt'] = x['RIAD4643'] - x['RIAD4627']
    # ncoOthLn contains ncoAg for small banks:  must be removed:
    def OthLnCalc(x):
      adj = 0
      if x['RIAD4645'] == 0:
        adj = x['RIAD4655'] - x['RIAD4665']
      return x['RIAD4644'] - x['RIAD4628'] - adj

    df['ncoOthLn'] = x.apply(OthLnCalc, axis = 1)

    try:
      df['ncoLeas'] = x.apply(lambda x: max(x['RIAD4266'],(x['RIADF185']+x['RIADC880'])) -\
                              max(x['RIAD4267'],(x['RIADF187']+x['RIADF188'])),axis=1)
    except:
      df['ncoLeas'] = x.apply(lambda x: max(x['RIAD4266'],(x['RIAD4658']+x['RIAD4659'])) -\
                              max(x['RIAD4267'],(x['RIAD4668']+x['RIAD4669'])),axis=1)

    #################################################################
    # add the "as of" date column and change the index to be IDRSSD (ref #) and as-of date:
    df['AsOf'] = datetime(y,int(m),ddd[m])
    df = df.reset_index()
    df = df.set_index(['IDRSSD','AsOf'])
    #df.head()
    if yrmn == yrmn_beg:
      df2 = df
    else:
      df2 = pd.concat([df2, df])
    #################################################################
  print(y, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


2010 2016-05-07 11:53:47
2011 2016-05-07 11:54:40
2012 2016-05-07 11:55:35
2013 2016-05-07 11:56:31
2014 2016-05-07 11:57:26
2015 2016-05-07 11:58:19


In [72]:
x2.head(2)

,RCONK049,RCONK050,RCONK051,RCONK052,RCONK053,RCONK054,RCONK055,RCONK056,RCONK057,RCONK058,...,RCONK286,RCONK287,RCONK288,RCONL183,RCONL184,RCONL185,RCONL186,RCONL187,RCONL188,Unnamed: 120
IDRSSD,,,,,,,,,,,,,,,,,,,,,
37.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
242.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [73]:
df2.to_csv("df2_" + str(yst) + "_" + str(yen-1) + ".csv")
len(df2) # number of data rows in extract table # 291793 (2001-2009), 165106 (2010-2015)

165106

In [74]:
"df2_" + str(yst) + "_" + str(yen-1) + ".csv"  # name of output file (Note: end year is 1 less than yen value!!)

'df2_2010_2015.csv'

In [75]:
df2.tail(2)

,,Name,State,Cert,eTotAst,eTotLn,eALLL,eOREO,eGW,eOthInt,eTotEq,...,ncoFIfor,ncoAg,ncoCIdom,ncoCIfor,ncoCard,ncoAuto,ncoOthCsmr,ncoForGovt,ncoOthLn,ncoLeas
IDRSSD,AsOf,,,,,,,,,,,,,,,,,,,,,
4536084,2015-12-31,BANK OF BIRD-IN-HAND,PA,59074,119957.0,104643.0,1299.0,0.0,0.0,0.0,14351.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4845861,2015-12-31,PRIMARY BANK,NH,59086,36441.0,7683.0,100.0,0.0,0.0,0.0,28089.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
